In [1]:

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from dateutil.parser import parse
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPRegressor
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import ADASYN
from collections import Counter

In [2]:

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential, load_model, save_model, Model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import AUC
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf


In [4]:
random_state = 42

In [5]:
feature_set =  'feature_set_normalised_pca'
X_train = pd.read_csv( F'./data/{feature_set}/X_train_full.csv')
y_train = pd.read_csv( F'./data/{feature_set}/y_train.csv').values.ravel()

X_val = pd.read_csv( F'./data/{feature_set}/X_valid_full.csv')
y_val = pd.read_csv( F'./data/{feature_set}/y_valid.csv').values.ravel()

In [6]:
# X_train_subset = X_train[['CUSTOMER_ID', 'TERMINAL_ID', 'TX_AMOUNT', 'Time_of_day_00:00-05:59',
#        'Time_of_day_06:00-11:59', 'Time_of_day_12:00-17:59',
#        'Time_of_day_18:00-23:59']]

# X_val_subset = X_val[['CUSTOMER_ID', 'TERMINAL_ID', 'TX_AMOUNT', 'Time_of_day_00:00-05:59',
#        'Time_of_day_06:00-11:59', 'Time_of_day_12:00-17:59',
#        'Time_of_day_18:00-23:59']]

In [7]:
# # outputing y value before over sampling.
# counter = Counter(y_train)
# print("Before Sampling: {}".format(counter))

# # Oversample using Adaptive Synthetic (ADASYN) algorithm.
# sm = ADASYN()
# # esample the dataset.
# xtrain_sm, ytrain_sm = sm.fit_resample(X_train, y_train)

# # outputing y value after over sampling.
# counter = Counter(ytrain_sm)
# print("After Sampling: {}".format(counter))

In [8]:
tf.random.set_seed(random_state)

model = tf.keras.Sequential([
#     tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy', tf.keras.metrics.Precision(),
            tf.keras.metrics.Recall()]
)

In [9]:
history = model.fit(
    X_train, y_train, epochs=200, verbose=1, validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,
                                               verbose=1, restore_best_weights=True)]
)

Epoch 1/200
18663/18663 [==============================] - 41s 2ms/step - loss: 0.1211 - accuracy: 0.9740 - precision: 0.1154 - recall: 1.9381e-04 - val_loss: 0.1183 - val_accuracy: 0.9745 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/200
18663/18663 [==============================] - 41s 2ms/step - loss: 0.1201 - accuracy: 0.9741 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.1182 - val_accuracy: 0.9745 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/200
18663/18663 [==============================] - 40s 2ms/step - loss: 0.1198 - accuracy: 0.9741 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.1182 - val_accuracy: 0.9745 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/200
18663/18663 [==============================] - 37s 2ms/step - loss: 0.1196 - accuracy: 0.9741 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.1184 - val_accuracy: 0.9745 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 5/200
18663/18

KeyboardInterrupt: 

In [25]:
model.predi

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 200)               24400     
                                                                 
 dense_13 (Dense)            (None, 200)               40200     
                                                                 
 dense_14 (Dense)            (None, 1)                 201       
                                                                 
Total params: 64801 (253.13 KB)
Trainable params: 64801 (253.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
# Returns the loss value & metrics values for the model in train mode.
acc = model.evaluate(X_train, y_train, verbose=0)
print("Train Accuracy: {:.2f}".format(acc*100))

# Returns the loss value & metrics values for the model in test mode.
acc = model.evaluate(X_val, y_val, verbose=0)
print("Test Accuracy: {:.2f}".format(acc*100))

# tf.debugging.set_log_device_placement(True)

TypeError: unsupported format string passed to list.__format__

In [29]:
train_pred = model.predict(X_train)

23329/23329 [==============================] - 28s 1ms/step


In [33]:
roc_auc_score(y_train,train_pred)

0.623172953522851

In [34]:
val_pred = model.predict(X_val)

9998/9998 [==============================] - 16s 2ms/step


In [35]:
roc_auc_score(y_val,val_pred)

0.5882404450536723